# Import libraries

In [1]:
import pypots
import os
import sys
from pypots.nn.functional import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd    
from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012 as preprocess_physionet2012
from missingData.toolkits import toolkits

2025-03-21 21:42:23.911006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742604143.927092  406705 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742604143.932040  406705 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 21:42:23.949088: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/pypots/nn/functional/cuda.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is


████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



# Loading database

### Standard Scaler 

In [4]:
set_random_seed()
physionet2012_dataset_standard = preprocess_physionet2012(subset="all", rate=0.1, normalization=1)
print(physionet2012_dataset_standard.keys())

2025-03-21 21:44:56 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-03-21 21:44:56 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-03-21 21:44:56 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-03-21 21:44:56 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-03-21 21:44:56 [INFO]: Loaded successfully!
2025-03-21 21:45:10 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-21 21:45:10 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-21 21:45:10 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [5]:
scaler_standard = physionet2012_dataset_standard["scaler"]

In [6]:
dataset_for_training_standard, dataset_for_validating_standard, dataset_for_testing_ori_standard, dataset_for_testing_standard = toolkits.separating_dataset(physionet2012_dataset_standard)

# Creating indicating mask and get test_X_ori

### Standard Scaler

In [7]:
dataset_for_testing_ori_standard_norm = toolkits.dict_to_list(dataset_for_testing_ori_standard)
dataset_for_testing_standard_norm = toolkits.dict_to_list(dataset_for_testing_standard)
indicating_mask_variable_standard_norm, test_X_ori_variable_standard_norm = toolkits.components_mae(dataset_for_testing_ori_standard_norm, dataset_for_testing_standard_norm)
indicating_mask_variable_standard_norm = toolkits.pre_reshape(indicating_mask_variable_standard_norm)
test_X_ori_variable_standard_norm = toolkits.pre_reshape(test_X_ori_variable_standard_norm)
indicating_mask_variable_standard_norm = toolkits.reshape_variable(indicating_mask_variable_standard_norm)
test_X_ori_variable_standard_norm = toolkits.reshape_variable(test_X_ori_variable_standard_norm)

# Inicialize the models

### SAITS

<h4> Inicialize new model (Standard)</h4>

In [10]:
saits_standard = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-21 21:51:42 [INFO]: No given device, using default device: cpu
2025-03-21 21:51:42 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-21 21:51:43 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


# Train/Load the models

### SAITS

<h4> Load - Standard Scaler </h4>

In [11]:
saits_standard.load("tutorial_results/imputation/saits/standard/SAITS.pypots")

2025-03-21 21:51:46 [INFO]: Model loaded successfully from tutorial_results/imputation/saits/standard/SAITS.pypots


# Testing stage

### SAITS

#### Standard Scaler

In [12]:
saits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, saits_standard)
saits_imputation_standard = toolkits.pre_reshape(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.desnormalization(saits_imputation_standard, scaler_standard)
saits_imputation_standard = toolkits.reshape_variable(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.reshape_variable(saits_imputation_standard_ori)

# Calculate mean absolute error

### SAITS

#### Standard scaler (C/Normalização)

In [13]:
testing_mae_saits_variables_standard = toolkits.calculate_mae(saits_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

# Results mean absolute error

In [14]:
subgroups = ["General", "Female", "Male", "Undefined Gender", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

In [15]:
variables = ["ALP", "ALT", "AST", "Albumin", "BUN", "Bilirubin", "Cholesterol", "Creatinine", "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT", "HR", "K", "Lactate", "MAP", "MechVent", "Mg", "NIDiasABP", "NIMAP", "NISysABP", "Na", "PaCO2", "PaO2", "Platelets", "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT", "Urine", "WBC", "Weight", "Ph"]

#### SAITS

<h4>Standard Scaler (C/Normalização)</h4>

In [16]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_standard, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 0.35549184664706085
ALT : 0.34616397235246166
AST : 0.283203929613111
Albumin : 0.5865378684716107
BUN : 0.23549456829210566
Bilirubin : 0.29213390018033225
Cholesterol : 0.8619142944054452
Creatinine : 0.2272360108358054
DiasABP : 0.25696352027850733
FiO2 : 0.4843457493583031
GCS : 0.27782594851598286
Glucose : 0.5574566141755575
HCO3 : 0.42235495223366226
HCT : 0.5478476366595461
HR : 0.2786515469045242
K : 0.6173738181988982
Lactate : 0.3602081488093162
MAP : 0.276148040474521
MechVent : 0.016131509283529688
Mg : 0.5428868319523208
NIDiasABP : 0.23805346595387422
NIMAP : 0.17731126967659114
NISysABP : 0.3059993623684565
Na : 0.5094466185029012
PaCO2 : 0.5448751678244528
PaO2 : 0.5044175503727509
Platelets : 0.37910753219049526
RespRate : 0.4975549864022565
SaO2 : 0.395203639666887
SysABP : 0.2876315375696396
Temp : 0.3654739421516718
TroponinI : 0.5742820858879293
TroponinT : 0.562502744207675
Urine : 0.4555801478350003
WBC : 0.15